In [1]:
from __future__ import division
import itertools

import numpy as np
from scipy import linalg
from scipy.stats import multivariate_normal
from sklearn import mixture
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [36]:
err = 0.01
N_GAUSSIANS = 5

data = np.genfromtxt('sampled_items.csv', delimiter='|')
data = np.delete(data, 2, 1)

print data.shape

gmm = mixture.GaussianMixture(n_components=N_GAUSSIANS, max_iter=500, reg_covar=1e-06).fit(data)
print gmm

mu = gmm.means_
sigma = gmm.covariances_
pi = gmm.weights_

(5000, 4)
GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=500,
        means_init=None, n_components=5, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)


In [45]:
def find_min(data, completeRangeJ):
    errs = np.abs((completeRangeJ - data)/ data)
    min_err = min(errs)
    return min_err, np.where(errs == min_err)[0][0]

In [53]:
n = data.shape[0]
m = data.shape[1]

compact = np.zeros((n, m))
totalErrs = np.zeros((n, m))

t = np.zeros((n, N_GAUSSIANS))
mub = np.zeros((n, m, N_GAUSSIANS))
xb = np.zeros((n, m))

minDifs = np.zeros((n, m))
minDifIdxs = np.array([[-1 for j in range(m)] for i in range(n)])

totalRems = 0

for i in range(n):
    print '\r',
    print i,
    
    rangeJ = np.array([j for j in range(m) if j not in minDifIdxs[i]])
    nRem = 0
    
    while(True):
    
        r_mu = [mu[k][rangeJ] for k in range(N_GAUSSIANS)]
        r_sigma = [sigma[k][rangeJ][rangeJ] for k in range(N_GAUSSIANS)]

        mvns = [multivariate_normal(mean, cov) for mean, cov in zip(r_mu, r_sigma)]

        for k in range(N_GAUSSIANS):
            num = pi[k]*mvns[k].pdf(data[i][rangeJ])
            den = sum([pi[pos]*mvns[pos].pdf(data[i][rangeJ]) for pos in range(N_GAUSSIANS)])

            t[i, k] = num/den
            for j in range(m):
                all_but_j = np.arange(m) != j
                mub[i, j, k] = mu[k, j] + \
                                np.dot(np.dot(np.delete(sigma[k, j], j), \
                                linalg.pinv(np.delete(sigma[k, all_but_j], j, 1))), \
                                (data[i, all_but_j] - mu[k, all_but_j]))

        xb[i, :] = sum([t[i, k]*mub[i, :, k] for k in range(N_GAUSSIANS)])
        
        completeRangeJ = xb[i, :]
        completeRangeJ[np.setdiff1d(np.arange(m), rangeJ)] = np.nan
        
        minDif, minDifIndex = find_min(completeRangeJ, data[i])
        minDifs[i, nRem] = minDif
        minDifIdxs[i, nRem] = minDifIndex
        nRem += 1
        isErr = False
        
        for j2 in [j for j in minDifIdxs[i] if j != -1]:
            erroJ2 = np.abs((xb[i,j2] - data[i,j2])/ data[i,j2])
            limErr = data[i, j2]*err
            if erroJ2 > limErr:
                isErr = True
        
        if nRem >= m or isErr:
            break
        
        compact[i] = minDifIdxs[i]
        totalErrs[i] = minDifs[i]
        totalRems += 1
    
print totalRems
    

4999 11178


In [55]:
totalRems/(n*m)

0.5589

nan